In [16]:
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from toolbox import ToolBox
from textblob import TextBlob

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Jules/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores('This is not super fun')

{'neg': 0.661, 'neu': 0.339, 'pos': 0.0, 'compound': -0.7048}

In [3]:
tb = ToolBox()

In [41]:
reviews = tb.load_data_sql(table='critic_reviews', where="lang = 'en'", limit=1000)
reviews.head()

,game,source,date,grade,review,lang
0,007-legends-pc,GamingXP,2012-12-03,35.0,"Unfortunately the game proves the fact, that d...",en
1,007-legends-pc,LEVEL (Czech Republic),2012-12-21,30.0,The worst gift agent 007 could possibly get to...,en
2,007-legends-pc,PC Gamer,2013-01-05,16.0,"Legends, you're the worst. [Feb 2013, p.79]",en
3,007-legends-pc,PC Master (Greece),2013-02-06,28.0,"Why put yourself through this ordeal, when the...",en
4,007-legends-pc,Riot Pixels,2013-01-29,25.0,007 Legends greets you with a warning that Act...,en


In [42]:
review = reviews.iloc[0]['review']

TextBlob(review).sentiment

Sentiment(polarity=0.016666666666666653, subjectivity=0.4761904761904762)

In [49]:
import flair
from flair.data import Sentence

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2019-10-02 14:28:03,804 loading file /Users/Jules/.flair/models/imdb-v0.4.pt


In [88]:
review = reviews[reviews['grade'] > 90].iloc[12]['review']

print(review)

s = flair.data.Sentence(review)
flair_sentiment.predict(s)

The only reason why we didnâ<U+0080><U+0099>t give our best vote to this game is because it is closely related to an event after which the title could lose part of its attractiveness (there are no league teams and there is no substantial change in the multiplayer mode). Other than this, the best just got better! FIFA World Cup 2010 is just a few inches from perfection.


[Sentence: "The only reason why we didnâ<U+0080><U+0099>t give our best vote to this game is because it is closely related to an event after which the title could lose part of its attractiveness (there are no league teams and there is no substantial change in the multiplayer mode). Other than this, the best just got better! FIFA World Cup 2010 is just a few inches from perfection." - 66 Tokens]

In [61]:
tagger = flair.models.SequenceTagger.load('nl-ner')

2019-10-02 14:43:39,071 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll2002-dutch/nl-ner-conll02-v0.1.pt not found in cache, downloading to /var/folders/h6/c23ktx711x517m98bmjdb8jh0000gn/T/tmp6n0clre3


100%|██████████| 1392544535/1392544535 [01:41<00:00, 13693478.51B/s]

2019-10-02 14:45:20,972 copying /var/folders/h6/c23ktx711x517m98bmjdb8jh0000gn/T/tmp6n0clre3 to cache at /Users/Jules/.flair/models/nl-ner-conll02-v0.1.pt


2019-10-02 14:45:22,915 removing temp file /var/folders/h6/c23ktx711x517m98bmjdb8jh0000gn/T/tmp6n0clre3
2019-10-02 14:45:23,022 loading file /Users/Jules/.flair/models/nl-ner-conll02-v0.1.pt


In [76]:
sentence = Sentence('Eetzaal van de aan het klooster verbonden kweekschool. De meisjes die daar werden opgeleid tot onderwijzeres verbleven intern in een internaat. De enige versieringen in deze zaal bestaan uit religieuze voorstellingen, waaronder een Mariabeeld en een H. Hartbeeld.')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
# print(sentence.to_tagged_string())

[Sentence: "Eetzaal van de aan het klooster verbonden kweekschool. De meisjes die daar werden opgeleid tot onderwijzeres verbleven intern in een internaat. De enige versieringen in deze zaal bestaan uit religieuze voorstellingen, waaronder een Mariabeeld en een H. Hartbeeld." - 38 Tokens]

In [77]:
for entity in sentence.get_spans('ner'):
    print(entity)

MISC-span [34]: "Mariabeeld"
MISC-span [37,38]: "H. Hartbeeld."


In [87]:
raw = "Sint-Janssingel gezien in zuidelijke richting ter hoogte van de Stationsbrug, later Wilhelminabrug. Midden op de achtergrond de molen bij het Wilhelminapark. Rechts de nieuwe bebouwing aan de Kloostersingel, hoek Luybenstraat. De rest van de nieuwe wijk het Zand is nog onbebouwd. Achter de bebouwing is de kerk van de paters Kapucijnen te zien. Links de Sint-Janssingel met daarachter het kloostercomplex van de Marienburg. Fotograaf is Herman de Ruijter HdR"

sentences = [Sentence(sentence) for sentence in raw.split('.')]

for sentence in sentences:
    print(sentence)
    tagger.predict(sentence)
    
    for entity in sentence.get_spans('ner'):
        print(entity)
        
    print('--------------------------------')

Sentence: "Sint-Janssingel gezien in zuidelijke richting ter hoogte van de Stationsbrug, later Wilhelminabrug" - 12 Tokens
LOC-span [1]: "Sint-Janssingel"
LOC-span [10]: "Stationsbrug,"
LOC-span [12]: "Wilhelminabrug"
--------------------------------
Sentence: "Midden op de achtergrond de molen bij het Wilhelminapark" - 9 Tokens
LOC-span [9]: "Wilhelminapark"
--------------------------------
Sentence: "Rechts de nieuwe bebouwing aan de Kloostersingel, hoek Luybenstraat" - 9 Tokens
LOC-span [7]: "Kloostersingel,"
LOC-span [9]: "Luybenstraat"
--------------------------------
Sentence: "De rest van de nieuwe wijk het Zand is nog onbebouwd" - 11 Tokens
LOC-span [8]: "Zand"
--------------------------------
Sentence: "Achter de bebouwing is de kerk van de paters Kapucijnen te zien" - 12 Tokens
PER-span [10]: "Kapucijnen"
--------------------------------
Sentence: "Links de Sint-Janssingel met daarachter het kloostercomplex van de Marienburg" - 10 Tokens
LOC-span [3]: "Sint-Janssingel"
LOC-sp